In [1]:
# install missing packages
%pip install plotly
%pip install py-cpuinfo
%pip install pandas-datareader
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# check system details
import os
import psutil
import cpuinfo

ram_info = psutil.virtual_memory()
print(f"Total RAM: {ram_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available RAM: {ram_info.available / 1024 / 1024 / 1024:.2f} GB")
print(f"Used RAM: {ram_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of RAM: {ram_info.percent}%")
print(f"CPU Cores: {os.cpu_count()}")
print(f"CPU Speed: {cpuinfo.get_cpu_info()['hz_actual_friendly']}")
disk_info = psutil.disk_usage(os.getcwd())
print(f"Total Disk: {disk_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available Disk: {disk_info.free / 1024 / 1024 / 1024:.2f} GB")
print(f"Used Disk: {disk_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of Disk: {disk_info.percent}%")

Total RAM: 15.47 GB
Available RAM: 14.17 GB
Used RAM: 1.03 GB
Percentage Usage Of RAM: 8.4%
CPU Cores: 4
CPU Speed: 2.5000 GHz
Total Disk: 24.99 GB
Available Disk: 12.49 GB
Used Disk: 12.50 GB
Percentage Usage Of Disk: 50.0%


In [3]:
# import requirements
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from nnet_regressor import Regression

[nltk_data] Downloading package vader_lexicon to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
2023-10-28 21:11:18.281898: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# get the data
riders = pd.read_csv("riders.csv")
riders = riders.sort_values(by=["date", "route", "daytype"]).reset_index(drop=True)

In [5]:
# only use 50% of the data
riders = riders.head(int(0.5 * riders.shape[0]))

In [6]:
# resort the data
riders = riders.sort_values(by=["route", "daytype", "date"]).reset_index(drop=True)

In [7]:
# lag the output variable
riders["rides(t-1)"] = riders.groupby(["route", "daytype"])["rides"].shift(1)
riders["rides(t-2)"] = riders.groupby(["route", "daytype"])["rides"].shift(2)
riders["rides(t-3)"] = riders.groupby(["route", "daytype"])["rides"].shift(3)
riders["rides(t-4)"] = riders.groupby(["route", "daytype"])["rides"].shift(4)

In [8]:
# remove rows with missing values
riders = riders.dropna().reset_index(drop=True)

In [9]:
# resort the data
riders = riders.sort_values(by=["date", "route", "daytype"]).reset_index(drop=True)

In [10]:
# get the testing data
y = riders[["rides"]]
X = riders.drop(columns="rides")
testX = X.tail(int(0.2 * X.shape[0])).reset_index(drop=True)
testy = y.tail(int(0.2 * y.shape[0])).reset_index(drop=True)

In [11]:
# build the model
print("\n---- Riders Regression Analysis ----\n")
model = Regression(
    name="Tensorflow Without Feature Engineering", 
    path=None,
    rename=True, 
    deep=True,
    time=True, 
    text=False,
    binary=True, 
    imputation=False, 
    variance=True,
    scale=True,
    atwood=False,
    binning=False,
    reciprocal=False, 
    interaction=False, 
    selection=False,
    tune=False,
    plots=True,
)
try:
    model.load()  # load the machine learning pipeline
    predictions = model.predict(testX)
except:
    model.explore(riders)
    model.validate(X, y)  # build the machine learning pipeline
    predictions = model.predict(testX)
    print("\nModel Performance:")
    print(f"R2: {model.r2}")
    print(f"RMSE: {model.rmse}")
    print(f"In Control: {model.in_control}")


---- Riders Regression Analysis ----

Visualizing The Data:
> Plotting Correlations
> rides vs. rides(t-1)
> rides vs. rides(t-2)
> rides vs. rides(t-3)
> rides vs. rides(t-4)
> rides(t-1) vs. rides(t-2)
> rides(t-1) vs. rides(t-3)
> rides(t-1) vs. rides(t-4)
> rides(t-2) vs. rides(t-3)
> rides(t-2) vs. rides(t-4)
> rides(t-3) vs. rides(t-4)
> Plotting rides
> Plotting rides(t-1)
> Plotting rides(t-2)
> Plotting rides(t-3)
> Plotting rides(t-4)
> Plotting daytype
> rides vs. route
> rides vs. daytype
> rides(t-1) vs. route
> rides(t-1) vs. daytype
> rides(t-2) vs. route
> rides(t-2) vs. daytype
> rides(t-3) vs. route
> rides(t-3) vs. daytype
> rides(t-4) vs. route
> rides(t-4) vs. daytype
28.75 Seconds
Model Training:
> Transforming The Training Data
> Renaming Features
> Extracting Time Features
> Transforming Categorical Features
> Renaming Features
> Removing Constant Features
> Scaling Features
> Training Neural Network


2023-10-28 21:12:28.510931: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 872573616 exceeds 10% of free system memory.


KeyboardInterrupt: 

In [ ]:
# model diagnostics
print("Model Indicators:")
for i, indicator in enumerate(model.indicators["Indicator"][:10].tolist()):
    print(f"{i+1}. {indicator}")
print(" ")
print("Feature Drift:")
for i, feature in enumerate(model.drift.loc[model.drift["pvalue"] < 0.05, "Feature"][:10].tolist()):
    print(f"{i+1}. {feature}")
if model.drift.loc[model.drift["pvalue"] < 0.05].shape[0] == 0:
    print("None")

In [ ]:
# score the model
rmse = mean_squared_error(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
    squared=False,
)
r2 = r2_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
)

print(f"RMSE: {rmse}")
print(f"R2: {r2}")

In [ ]:
# save the machine learning pipeline
model.dump()

In [ ]:
# refit the model to include the test data
model.refit(testX, testy)